In [2]:
from experiments.model_training.train_qwen import train

/home/ubuntu/miniconda3/envs/tau2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-05 11:55:35.502 | INFO     | tau2.utils.utils:<module>:27 - Using data directory from source: /lambda/nfs/victor-north-tx/tau2-bench-private/data
2025-08-05 11:55:36.722 | INFO     | tau2.utils.llm_utils:<module>:43 - VLLM_API_BASE: http://127.0.0.1:8000/v1
2025-08-05 11:55:36.722 | INFO     | tau2.utils.llm_utils:<module>:44 - OLLAMA_API_BASE: http://127.0.0.1:11434
2025-08-05 11:55:36.723 | INFO     | tau2.utils.llm_utils:<module>:78 - LiteLLM: Cache is disabled
2025-08-05 11:55:36.723 | WARNING  | tau2.utils.llm_utils:<module>:85 - Sonnet thinking is disabled
2025-08-05 11:55:36.797 | DEBUG    | tau2.registry:<module>:194 - Registering default components...
2025-08-05 11:55:36.798 | DEBUG    | tau2.registry:<mod

In [ ]:
from pathlib import Path

model_name = "Qwen2.5-3B-instruct"
qwen_model = f"/home/ubuntu/victor-north-tx/models/{model_name}"
train_dataset_path = "data/train_full-v1.jsonl"
test_dataset_path = "data/test_full-v1.jsonl"
trained_model_name = f"{model_name}-sft-full-tau2-v1"
output_dir = f"./data/{trained_model_name}"
report_to_wandb = True
run_name = f"{trained_model_name}"
if Path(output_dir).exists():
    raise ValueError(f"Output directory {output_dir} already exists. Please delete it or choose a different output directory.")

max_train_datapoints = None
max_test_datapoints = None
use_peft = False
patience = 2

train(
    qwen_model=qwen_model, 
    train_dataset_path=train_dataset_path, 
    test_dataset_path=test_dataset_path, 
    output_dir=output_dir, 
    max_train_datapoints=max_train_datapoints, 
    max_test_datapoints=max_test_datapoints, 
    use_peft=use_peft,
    patience=patience,
    report_to_wandb=report_to_wandb,
    run_name=run_name
    )

Train dataset length: 1722
Test dataset length: 1107
Loading tokenizer and model for /home/ubuntu/victor-north-tx/models/Qwen2.5-3B-instruct
Tokenizer has a chat template.
Tokenizer eos_token_id: 151645, pad_token_id: 151643


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.22s/it]


Model loaded in 12.00 seconds


Truncating eval dataset: 100%|██████████| 1107/1107 [00:00<00:00, 9212.11 examples/s] 
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,0.001300,0.001436
2,0.001400,0.001415
3,0.001500,0.001359


In [2]:
from experiments.model_training.train_qwen import get_model_and_tokenizer
model, tokenizer = get_model_and_tokenizer(model_name=output_dir)
model


/home/ubuntu/miniconda3/envs/tau2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-05 12:12:24.497 | INFO     | tau2.utils.utils:<module>:27 - Using data directory from source: /lambda/nfs/victor-north-tx/tau2-bench-private/data
2025-08-05 12:12:25.726 | INFO     | tau2.utils.llm_utils:<module>:43 - VLLM_API_BASE: http://127.0.0.1:8000/v1
2025-08-05 12:12:25.727 | INFO     | tau2.utils.llm_utils:<module>:44 - OLLAMA_API_BASE: http://127.0.0.1:11434
2025-08-05 12:12:25.728 | INFO     | tau2.utils.llm_utils:<module>:78 - LiteLLM: Cache is disabled
2025-08-05 12:12:25.728 | WARNING  | tau2.utils.llm_utils:<module>:85 - Sonnet thinking is disabled
2025-08-05 12:12:25.807 | DEBUG    | tau2.registry:<module>:194 - Registering default components...
2025-08-05 12:12:25.807 | DEBUG    | tau2.registry:<mod

NameError: name 'output_dir' is not defined

In [3]:
from experiments.model_training.train_qwen import get_model_and_tokenizer, QWEN25_TEMPLATE, check_chat_template_supports_assistant_only_loss

model_name = "Qwen2.5-3B-instruct"
qwen_model = f"/home/ubuntu/victor-north-tx/models/{model_name}"
tokenizer = get_model_and_tokenizer(qwen_model)[1]
tokenizer.chat_template = QWEN25_TEMPLATE
tokenizer.eos_token
tokenizer.eos_token_id
tokenizer.pad_token_id
tokenizer.chat_template

print(check_chat_template_supports_assistant_only_loss(tokenizer))

Loading tokenizer and model for /home/ubuntu/victor-north-tx/models/Qwen2.5-3B-instruct
Tokenizer has a chat template.
Tokenizer eos_token = <|im_end|>
Tokenizer eos_token_id: 151645, pad_token_id: 151643
Tokenizer chat_template supports assistant only loss: False


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]

Model loaded in 7.04 seconds
True


In [9]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."}
]

res = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, add_generation_prompt=False, return_assistant_tokens_mask=True)
print(res)
[tokenizer.decode(token_id) for i, token_id in enumerate(res['input_ids']) if res['assistant_masks'][i]]

{'input_ids': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 3838, 374, 279, 6722, 315, 9625, 30, 151645, 198, 151644, 77091, 198, 785, 6722, 315, 9625, 374, 12095, 13, 151645, 271], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


['<|im_start|>',
 'assistant',
 '\n',
 'The',
 ' capital',
 ' of',
 ' France',
 ' is',
 ' Paris',
 '.',
 '<|im_end|>',
 '\n\n']